In [1]:
using LinearAlgebra
using Symbolics
using MultivariatePolynomials
using DynamicPolynomials
using HomotopyContinuation
using CairoMakie                 # para plots
using Distributions
using Random

# Função que arredonda polinômios

function roundp(p)
      if p isa AbstractPolynomialLike
          cs = coefficients(p)
          ms = monomials(p)
          return isempty(cs) ? zero(p) : sum(round(c; digits=1) * m for (c, m) in zip(cs, ms))
      elseif p isa Number
          return round(p; digits=2)
      elseif p isa AbstractArray
          return map(roundp, p)
      else
          error("Unsupported type in roundp: $(typeof(p))")
      end
  end
  # ---------

roundp (generic function with 1 method)

In [56]:
# Parâmetros:

n = 3
k = 2
p = 1
q = p + 1

m1 = q * q
m2 = k * q ;

In [57]:
gamma_0 = collect(1:p)                     # coluna p×1
b_0     = collect(2:(k*p)+1)

Gamma_0 = [ 1.0           zeros(1,p) ;     # 1×(1+p)
            gamma_0       I(p)      ]       # p×1   p×p  -> (1+p)×(1+p)

B_0 = [ zeros(k,1)  reshape(b_0, k, p) ]

Σ = Matrix{Float64}(I, q, q)  

Random.seed!(123)

U_0 = rand( MvNormal(zeros(q) , Σ  ) , n  )'
X = rand(n, k)


Y = X * B_0 * inv(Gamma_0) + U_0*inv(Gamma_0) ; 



In [58]:
# matrizes de restrição

R1 = [ ones(1, 1)                 zeros(1, p)                         zeros(1, p*q);
       zeros(p, 1)              zeros(p, p)                       zeros(p, p*q);
       zeros(p*q, 1)          zeros(p*q, p)                   Matrix( I, p*q, p*q )  ]

R2 = [ Matrix(I, k, k)                 zeros(k, k*p);
       zeros(k*p, k)               zeros(k*p, k*p)  ]


4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [59]:
# Gamma e b

@polyvar γ[1:p] b[1:k*p]

Γ = [ 1                    zeros(1, p);
      γ                    Matrix(I, p, p) ]


B = [ zeros(k, 1)   reshape(b, k, p) ]        

U = Y * Γ - X * B        # n×p (simbólico)

# multiplicadores

@polyvar   μ1[ 1: q^2 - p ]   μ2[1:k]

lambda1 = [ μ1[1] ; zeros(p,1) ; μ1[2:end]  ]   
lambda2 = [  μ2[1:k] ; zeros(k*p , 1) ]   

                          

4×1 Matrix{Term{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}}:
 μ2₁
 μ2₂
 0.0
 0.0

In [60]:
eqA = reshape(Y' * U, q*q, 1) +  kron(U' * U, Matrix(I, q, q)) * R1 * lambda1

eqB = reshape((X' * U)', k*q, 1) + kron(U' * U, Matrix(I, k, k)) * R2 * lambda2

# final

F = vec( [eqA ; eqB] ) 

vars = [ γ ; b ; μ1 ; μ2 ]


F_sys = System(F; variables=vars)


result = solve(F_sys ; start_system = :total_degree)

S = [ real.(round.(sol; digits=4)) for sol in solutions(result)  ]


Tracking 6561 paths... 100%|████████████████████████████| Time: 0:00:04
                   # paths tracked: 6561
   # non-singular solutions (real): 3 (3)
       # singular endpoints (real): 6 (0)
          # total solutions (real): 9 (3)


3-element Vector{Vector{Float64}}:
 [1.8627, 2.5588, -2.3057, -1.0, 1.8627, -1.0, -0.1617, -0.2162]
 [1.3685, 5.8247, -3.6072, -1.0, 1.3685, -1.0, -0.1481, -0.3921]
 [1.0994, -0.9282, 5.7165, -1.0, 1.0994, -1.0, 0.2251, -0.3737]

In [33]:
function Q(x)
   
    Gamma = [ 1 0 ;
              x[1] 1]
    
    B = [ 0 x[2] ]   

    
    U = Y*Gamma - X*B

    return  det(U'U)
end  


opt =   S[ argmin( [ Q(x) for x in S ]   ) ][1:2]   

LoadError: ArgumentError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

# Hausman

In [61]:
# Parâmetros:

n = 3
k = 2
p = 1
q = p + 1

m1 = q * q
m2 = k * q ;

gamma_0 = collect(1:p)                     # coluna p×1
b_0     = collect(2:(k*p)+1)

Gamma_0 = [ 1.0           zeros(1,p) ;     # 1×(1+p)
            gamma_0       I(p)      ]       # p×1   p×p  -> (1+p)×(1+p)

B_0 = [ zeros(k,1)  reshape(b_0, k, p) ]

Σ = Matrix{Float64}(I, q, q)  

Random.seed!(123)

U_0 = rand( MvNormal(zeros(q) , Σ  ) , n  )'
X = rand(n, k)


Y = X * B_0 * inv(Gamma_0) + U_0*inv(Gamma_0) ; 



W = [  Y[:, 2:q]           zeros(n , p*k) ;
       zeros(n*p , p )     kron(Matrix(I, p, p) , X  )        ]



6×3 Matrix{Float64}:
 -0.391902  0.0       0.0
  3.23003   0.0       0.0
  4.65079   0.0       0.0
  0.0       0.044818  0.327238
  0.0       0.933353  0.526996
  0.0       0.58056   0.836229

In [62]:


function iter(x)

    B2 = reshape(x[p+1:end], k, p)
    γ = x[1:p]
    
    What =  [  -X* B2         zeros(n , p*k) ;
             zeros(n*p , p )     kron(Matrix(I, p, p) , X  )        ]

    
    Γ = [ 1     zeros(1,p) ;
          γ  Matrix(I, p, p)    ]
    
    B = [ zeros(k,1)  B2   ]           
     

    S = (Y*Γ - X*B )' * (Y*Γ - X*B )

    G = What' * kron( inv(S)   , Matrix(I, n, n)  )

    return  inv(G * W ) * G * vec(Y)
end

iter([i for i in 1:p + k*p])

3-element Vector{Float64}:
 -1.1613548822916075
  3.284599189581961
  1.9209184095906942

In [66]:
# parâmetros da iteração
tol   = 1e-6
maxit = 1000

# chute inicial (ex.: p = k = 2  →  6 números)
x = [i for i in 1:(p + k*p)]
hausman   = nothing
convergiu = false

for it in 1:maxit
    y = iter(x)                                # próximo vetor

    if maximum(abs.(y .- x)) < tol             # critério de parada
        g2 = y[1:p]
        B2 = reshape(y[p+1:end], k, p)

        hausman = (-g2, B2)                    # salva: (−γ, B₂)
        convergiu = true
        println("Convergiu em $it iterações")
        break
    end
    x = y
end

if convergiu
else
    println("NÃO convergiu em $maxit iterações.")
end


hausman


Convergiu em 6 iterações


([1.164535836487564], [2.258872534601309; 2.9784848945917552;;])